In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_01049.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00778.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00560.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00429.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00445.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00754.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00228.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_real/real_00775.jpg
/kaggle/input/real-and-fake-face-detecti

/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/mid_310_0111.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/mid_165_0101.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/easy_101_0010.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/easy_133_1100.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/easy_129_1111.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/mid_102_1100.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/mid_254_1111.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face_detection/real_and_fake_face/training_fake/easy_4_0011.jpg
/kaggle/input/real-and

/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/hard_121_1100.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/hard_96_1110.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/mid_345_1111.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/easy_71_1100.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/hard_134_1100.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/mid_309_1111.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/hard_115_1110.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/mid_368_0110.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/mid_138_0011.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake/mid_419_0011.jpg
/kaggle/input/real-and-fake-face-detection/real_and_fake_face/train

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
from skimage.io import imread, imshow
import cv2
import matplotlib.pyplot as plt

In [36]:
import keras
from keras.applications import VGG19, VGG16, resnet50, InceptionV3
from tensorflow.keras.applications import EfficientNetB0
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inceptionv3
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficient_net
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.layers import Dense, Dropout
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

In [9]:
from PIL import Image
from skimage import img_as_float

In [10]:
## need this plugin to read png images
import skimage.io as io
io.use_plugin('pil', 'imread')

In [11]:
russel_crow_images = glob('/kaggle/input/lab3-datasets/russel_crowe_images/*.PNG') + glob('/kaggle/input/lab3-datasets/russel_crowe_images/*.jpg') 
people_images = glob('/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_real/*.PNG') + glob('/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_real/*.jpg') 

In [12]:
len(russel_crow_images), len(people_images)

(301, 1081)

In [13]:
df_path = pd.DataFrame(russel_crow_images + people_images, columns=['image_name'])

In [14]:
df_path['target'] = df_path.image_name.isin(russel_crow_images).astype(int)

In [15]:
df_path.head(3)

,image_name,target
0,/kaggle/input/lab3-datasets/russel_crowe_image...,1
1,/kaggle/input/lab3-datasets/russel_crowe_image...,1
2,/kaggle/input/lab3-datasets/russel_crowe_image...,1


In [11]:
train_images, test_images = train_test_split(df_path, test_size=0.3, stratify=df_path['target'])

In [12]:
train_images.head(3)

,image_name,target
955,/kaggle/input/real-and-fake-face-detection/rea...,0
66,/kaggle/input/lab3-datasets/russel_crowe_image...,1
1380,/kaggle/input/real-and-fake-face-detection/rea...,0


# VGG16 embeddings

In [13]:
#load VGG16 model, pretrained on ImageNet
model_vgg16 = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


### Create VGG16 embeddings for train images

In [14]:
vgg16_feats = []
for img_path in tqdm(train_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input_vgg16(img_data)

    vgg16_feature = model_vgg16.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feats.append(vgg16_feature_np.flatten())

100%|██████████| 967/967 [02:32<00:00,  6.34it/s]


In [15]:
vgg16_feats[0]

array([ 0.      ,  0.      ,  0.      , ..., 45.018017, 12.427753,
       24.422565], dtype=float32)

In [16]:
train_images = train_images.join(pd.DataFrame(np.array(vgg16_feats)))

In [17]:
train_images.head()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
955,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,57.007713,0.0,0.0,0.000000,0.000000,63.855167
66,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,52.81078,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,13.627598,0.0,0.0,0.000000,0.0,0.0,0.079455,10.876463,0.000000
1380,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,9.940681
906,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,27.888321,0.000000


### Create VGG16 embeddings for test images

In [18]:
vgg16_feats_test = []
for img_path in tqdm(test_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input_vgg16(img_data)

    vgg16_feature = model_vgg16.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feats_test.append(vgg16_feature_np.flatten())

100%|██████████| 415/415 [01:05<00:00,  6.38it/s]


In [19]:
vgg16_feats_test[0]

array([ 0.     ,  0.     ,  0.     , ...,  0.     , 10.96294,  0.     ],
      dtype=float32)

In [20]:
test_images = test_images.join(pd.DataFrame(np.array(vgg16_feats_test)))

In [21]:
test_images.tail()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
141,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.157674,0.0,0.0,0.0,0.0,8.866781,22.076271
168,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,22.614943,0.000000
1006,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
test_images[test_images.target == 0].dropna()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
353,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,45.926266,0.000000,0.000000,0.000000,6.621946,0.000000
327,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,36.626141,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.658529,0.000000,0.000000,0.000000,0.000000,24.429346
408,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.223794,45.856171,0.000000,0.000000,0.000000,0.000000,0.000000
303,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,66.062111,0.000000,0.000000,0.000000,2.854655,0.000000,0.000000,0.000000,0.000000
399,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.702980,5.420254,0.000000
324,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,18.867603,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.670821
329,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.946591
339,/kaggle/input/real-and-fake-face-detection/rea...,0,11.432357,0.0,0.000000,0.000000,0.0,0.000000,0.000000,15.012802,...,0.000000,0.000000,0.000000,0.000000,16.560337,0.000000,0.000000,0.000000,11.593105,0.000000
405,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,2.715199,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
361,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
train_images.head(2)

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
955,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,57.007713,0.0,0.0,0.000000,0.000000,63.855167
66,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,52.81078,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,13.627598,0.0,0.0,0.000000,0.0,0.0,0.079455,10.876463,0.000000


In [26]:
train_images.shape, test_images.shape

((967, 8194), (415, 8194))

In [27]:
train_images.to_csv('train_images_vgg16.csv', index=False)

In [28]:
test_images.to_csv('test_images_vgg16.csv', index=False)

# VGG19 embeddings

In [29]:
#load VGG19 model, pretrained on ImageNet
model_vgg19 = VGG19(weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


### Create VGG19 embeddings for train images

In [30]:
vgg19_feats = []
for img_path in tqdm(train_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input_vgg19(img_data)

    vgg19_feature = model_vgg19.predict(img_data)
    vgg19_feature_np = np.array(vgg19_feature)
    vgg19_feats.append(vgg19_feature_np.flatten())

100%|██████████| 967/967 [02:52<00:00,  5.60it/s]


In [31]:
vgg19_feats[0]

array([ 0.      ,  0.      ,  0.      , ..., 76.06525 , 32.628017,
        0.      ], dtype=float32)

In [32]:
train_images_vgg19 = train_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(vgg19_feats)))

In [33]:
train_images_vgg19.head()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
955,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.0,0.0,0.0,7.433889,0.0,0.811353,0.0,...,0.0,16.343136,0.0,0.0,20.089308,0.0,0.000000,0.0,0.000000,37.920971
66,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.119959,0.0,0.0,0.000000,0.0,0.219941,0.0,0.106321,0.000000
1380,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.0,0.0,0.0,1.272391,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
906,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,8.142830,0.0,0.000000,21.826845


### Create VGG19 embeddings for test images

In [34]:
vgg19_feats_test = []
for img_path in tqdm(test_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input_vgg19(img_data)

    vgg19_feature = model_vgg19.predict(img_data)
    vgg19_feature_np = np.array(vgg19_feature)
    vgg19_feats_test.append(vgg19_feature_np.flatten())

100%|██████████| 415/415 [01:14<00:00,  5.59it/s]


In [35]:
vgg19_feats_test[0]

array([0.       , 0.       , 0.       , ..., 0.       , 2.7075436,
       4.649713 ], dtype=float32)

In [36]:
test_images_vgg19 = test_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(vgg19_feats_test)))

In [37]:
test_images_vgg19.tail()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
141,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,4.618046,0.0,0.0,0.0,0.0,15.026509,0.000000
168,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.0,0.0,0.0,0.0,0.0,41.564449,0.0,...,0.0,0.0,57.821671,0.000000,0.0,0.0,0.0,0.0,0.000000,21.301191
1006,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
train_images_vgg19.shape, test_images_vgg19.shape

((967, 8194), (415, 8194))

In [39]:
train_images_vgg19.to_csv('train_images_vgg19.csv', index=False)

In [40]:
test_images_vgg19.to_csv('test_images_vgg19.csv', index=False)

# Resnet50 embeddings

In [47]:
# load Resnet50 model, pretrained on ImageNet
model_resnet50 = resnet50.ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


## Create Resnet50 embeddings for train

In [55]:
train_resnet50 = []
for img_path in tqdm(train_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet50.preprocess_input(x)
    resnet50_feature = model_resnet50.predict(x)
    resnet50_feature_np = np.array(resnet50_feature)
    train_resnet50.append(resnet50_feature_np.flatten())

100%|██████████| 967/967 [01:46<00:00,  9.06it/s]


In [57]:
train_resnet50[0], train_resnet50[0].shape

(array([0.      , 0.      , 0.      , ..., 0.      , 0.      , 4.605968],
       dtype=float32),
 (32768,))

In [58]:
train_images_resnet50 = train_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(train_resnet50)))

In [59]:
train_images_resnet50.head()

,image_name,target,0,1,2,3,4,5,6,7,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
955,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,3.044163,0.000000,0.000000,25.067244,0.543142,0.377944,0.000000,0.0,0.000000,1.006015
66,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.000000,0.000000,0.288248,1.581888,0.0,0.0,0.000000,0.657782,...,0.000000,0.000000,0.000000,0.030099,0.000000,0.000000,1.821778,0.0,0.744289,0.519091
1380,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.000000,0.000000,0.138810,0.0,0.0,0.204843,0.000000,...,0.000000,4.928546,0.462032,2.021258,0.000000,0.377606,0.000000,0.0,0.348176,0.000000
906,/kaggle/input/real-and-fake-face-detection/rea...,0,0.373176,0.279747,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,7.533114


## Create Resnet50 embeddings for test

In [60]:
test_resnet50 = []
for img_path in tqdm(test_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet50.preprocess_input(x)
    resnet50_feature = model_resnet50.predict(x)
    resnet50_feature_np = np.array(resnet50_feature)
    test_resnet50.append(resnet50_feature_np.flatten())

100%|██████████| 415/415 [00:44<00:00,  9.36it/s]


In [61]:
test_resnet50[0], test_resnet50[0].shape

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), (32768,))

In [62]:
test_images_resnet50 = test_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(test_resnet50)))

In [63]:
test_images_resnet50.head()

,image_name,target,0,1,2,3,4,5,6,7,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
353,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.0,4.588893,15.776166,0.664521,0.000000,0.0,0.0,0.0,10.174326
327,/kaggle/input/real-and-fake-face-detection/rea...,0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,11.620795,0.0,0.000000,20.473824,2.187519,0.000000,0.0,0.0,0.0,0.000000
122,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.000000,0.0,8.423420,0.0,0.0,3.399581,0.0,...,3.067380,0.0,0.000000,7.719499,0.555554,1.520991,0.0,0.0,0.0,0.000000
880,/kaggle/input/real-and-fake-face-detection/rea...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.0,0.416306,0.0,6.547116,0.0,0.0,0.000000,0.0,...,15.102789,0.0,0.000000,30.038370,0.000000,0.000000,0.0,0.0,0.0,0.691485


In [64]:
train_images_resnet50.to_csv('train_images_resnet50.csv', index=False)

In [65]:
test_images_resnet50.to_csv('test_images_resnet50.csv', index=False)

# InceptionV3 embeddings

In [21]:
# load InceptionV3 model, pretrained on ImageNet
model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


## Create InceptionV3 embeddings for train

In [26]:
train_inceptionv3 = []
for img_path in tqdm(train_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_inceptionv3(x)
    inceptionv3_feature = model_inceptionv3.predict(x)
    inceptionv3_feature_np = np.array(inceptionv3_feature)
    train_inceptionv3.append(inceptionv3_feature_np.flatten())

100%|██████████| 967/967 [01:18<00:00, 12.32it/s]


In [27]:
train_inceptionv3[0], train_inceptionv3[0].shape

(array([0.        , 0.17170693, 0.9086642 , ..., 0.4478378 , 3.567119  ,
        0.23508692], dtype=float32),
 (8192,))

In [28]:
train_images_inceptionv3 = train_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(train_inceptionv3)))

In [29]:
train_images_inceptionv3.head()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.171707,0.908664,0.216517,0.000000,0.587664,0.000000,0.930312,...,0.371986,0.000000,0.000000,3.292775,0.000000,1.843987,0.000000,0.447838,3.567119,0.235087
1,/kaggle/input/lab3-datasets/russel_crowe_image...,1,0.826839,1.426643,3.316109,0.000000,1.662804,0.319995,0.603856,6.109331,...,0.729552,0.000000,3.778265,1.857516,0.000000,1.443561,0.000000,0.000000,1.693114,1.821976
2,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,1.324509,0.000000,0.000000,0.000000,0.000000,0.000000,1.235800,...,0.000000,0.269610,0.000000,2.280240,0.049052,0.924208,0.000000,0.975079,2.287893,0.000000
3,/kaggle/input/real-and-fake-face-detection/rea...,0,0.329760,1.991935,1.426681,2.966591,0.000000,0.232661,0.999380,0.072799,...,1.405485,1.181545,0.714333,1.055129,0.028392,0.389198,0.416816,1.563948,1.957429,0.069018
4,/kaggle/input/real-and-fake-face-detection/rea...,0,0.238011,1.414896,0.000000,0.980220,0.000000,1.166587,0.000000,0.208673,...,4.255121,7.309142,1.263135,3.067232,0.000000,0.000000,0.000000,1.265768,3.846119,1.157779


## Create InceptionV3 embeddings for test

In [30]:
test_inceptionv3 = []
for img_path in tqdm(test_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_inceptionv3(x)
    inceptionv3_feature = model_inceptionv3.predict(x)
    inceptionv3_feature_np = np.array(inceptionv3_feature)
    test_inceptionv3.append(inceptionv3_feature_np.flatten())

100%|██████████| 415/415 [00:33<00:00, 12.39it/s]


In [31]:
test_inceptionv3[0], test_inceptionv3[0].shape

(array([0.       , 0.       , 0.       , ..., 1.3722637, 0.9622838,
        1.3184344], dtype=float32),
 (8192,))

In [32]:
test_images_inceptionv3 = test_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(test_inceptionv3)))

In [33]:
test_images_inceptionv3.head()

,image_name,target,0,1,2,3,4,5,6,7,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,/kaggle/input/real-and-fake-face-detection/rea...,0,0.000000,0.000000,0.000000,1.803032,0.000000,0.836092,1.185129,0.000000,...,1.990214,0.520279,3.712692,0.000000,2.383655,0.672592,1.031857,1.372264,0.962284,1.318434
1,/kaggle/input/real-and-fake-face-detection/rea...,0,1.636934,0.919219,0.000000,0.117485,0.000000,0.000000,0.000000,0.777001,...,0.000000,1.762136,0.585502,1.128088,0.000000,1.191162,0.000000,1.412112,0.446991,3.926535
2,/kaggle/input/lab3-datasets/russel_crowe_image...,1,1.894976,0.739461,0.669953,0.913079,0.000000,0.000000,0.394430,0.243897,...,0.985100,0.000000,0.711588,1.113838,0.000000,0.551073,0.000000,1.640609,0.452079,0.000000
3,/kaggle/input/real-and-fake-face-detection/rea...,0,2.683943,1.693666,0.401734,1.152178,0.000000,0.006503,0.000000,0.809075,...,1.438238,2.359625,0.000000,2.631158,0.651383,1.016426,1.858793,1.620718,0.656292,0.750156
4,/kaggle/input/lab3-datasets/russel_crowe_image...,1,1.263255,1.307458,4.097427,0.000000,2.940785,0.000000,0.139697,2.305952,...,0.000000,0.000000,2.086911,1.942440,2.338839,1.618610,0.000000,1.116177,0.286942,2.514364


In [34]:
train_images_inceptionv3.to_csv('train_images_inceptionv3.csv', index=False)

In [35]:
test_images_inceptionv3.to_csv('test_images_inceptionv3.csv', index=False)

# EfficientNet embeddings

In [37]:
# load EfficientNet model, pretrained on ImageNet
model_efficient_net = EfficientNetB0(weights='imagenet', include_top=False)

16711680/16705208 [==============================] - 0s 0us/step


## Create EfficientNet embeddings for train

In [38]:
train_efficient_net = []
for img_path in tqdm(train_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_efficient_net(x)
    efficient_net_feature = model_efficient_net.predict(x)
    efficient_net_feature_np = np.array(efficient_net_feature)
    train_efficient_net.append(efficient_net_feature_np.flatten())

100%|██████████| 967/967 [01:12<00:00, 13.27it/s]


In [39]:
train_efficient_net[0], train_efficient_net[0].shape

(array([-0.10880712, -0.06320287, -0.15144576, ..., -0.00373334,
        -0.04565866,  0.42348108], dtype=float32),
 (20480,))

In [40]:
train_images_efficent_net = train_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(train_efficient_net)))

In [41]:
train_images_efficent_net.head()

,image_name,target,0,1,2,3,4,5,6,7,...,20470,20471,20472,20473,20474,20475,20476,20477,20478,20479
0,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.108807,-0.063203,-0.151446,-0.258653,-0.143210,0.520563,-0.157612,-0.166628,...,-0.247935,-0.238898,-0.220627,-0.215591,-0.273874,-0.122365,-0.143420,-0.003733,-0.045659,0.423481
1,/kaggle/input/lab3-datasets/russel_crowe_image...,1,-0.224324,-0.275059,-0.275332,-0.273451,-0.181340,-0.120140,-0.275968,-0.053298,...,0.857371,-0.248763,3.022597,0.175123,-0.212562,-0.276527,-0.214430,-0.125209,-0.057769,-0.126134
2,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.143519,-0.052951,-0.136866,-0.270988,-0.116176,-0.217135,-0.141731,-0.233000,...,-0.235623,-0.144793,-0.139224,3.525116,2.044337,-0.034912,-0.206656,-0.025165,-0.030189,1.967990
3,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.276989,-0.234313,-0.074228,-0.158470,-0.159704,-0.266977,-0.092746,-0.066150,...,-0.099200,-0.132975,-0.090224,-0.100410,2.619948,-0.032482,-0.036605,-0.017058,-0.065093,-0.275775
4,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.148075,0.423263,-0.272181,-0.236746,-0.062076,-0.162534,-0.044187,-0.115152,...,-0.209433,-0.084877,0.664253,-0.131590,2.127666,-0.003567,-0.105610,-0.013514,-0.084880,-0.273628


## Create EfficientNet embeddings for test

In [42]:
test_efficient_net = []
for img_path in tqdm(test_images.image_name.values):
    img = image.load_img(img_path, target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_efficient_net(x)
    efficient_net_feature = model_efficient_net.predict(x)
    efficient_net_feature_np = np.array(efficient_net_feature)
    test_efficient_net.append(efficient_net_feature_np.flatten())

100%|██████████| 415/415 [00:31<00:00, 13.05it/s]


In [43]:
test_efficient_net[0], test_efficient_net[0].shape

(array([-0.20839457, -0.08960397, -0.27751622, ..., -0.12352552,
        -0.09803975,  0.30270344], dtype=float32),
 (20480,))

In [44]:
test_images_efficient_net = test_images[['image_name', 'target']].join(
    pd.DataFrame(np.array(test_efficient_net)))

In [45]:
test_images_efficient_net.head()

,image_name,target,0,1,2,3,4,5,6,7,...,20470,20471,20472,20473,20474,20475,20476,20477,20478,20479
0,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.208395,-0.089604,-0.277516,-0.278205,-0.278039,-0.139821,0.172332,-0.131331,...,-0.069444,-0.238976,-0.225213,-0.219753,0.102104,-0.018187,-0.173539,-0.123526,-0.098040,0.302703
1,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.262985,-0.122196,-0.107905,-0.264710,-0.173560,-0.252459,-0.200744,-0.078132,...,-0.076574,-0.234350,-0.084776,-0.268097,0.026403,-0.103383,-0.032805,-0.077539,-0.079280,-0.235686
2,/kaggle/input/lab3-datasets/russel_crowe_image...,1,-0.190629,-0.018511,-0.166034,-0.278226,-0.163346,-0.274915,-0.243579,-0.053505,...,-0.010374,-0.016908,-0.220223,-0.064096,-0.150552,-0.066105,-0.140162,-0.057970,-0.110853,-0.140567
3,/kaggle/input/real-and-fake-face-detection/rea...,0,-0.153155,-0.270896,-0.126786,-0.089576,-0.224660,-0.268549,-0.115283,-0.151899,...,-0.182359,-0.129820,-0.097907,-0.278464,-0.167320,-0.030293,-0.210425,-0.024222,-0.068577,-0.277990
4,/kaggle/input/lab3-datasets/russel_crowe_image...,1,-0.146110,-0.024241,-0.253659,0.050540,-0.027650,-0.274519,-0.277906,-0.274318,...,-0.064542,-0.073432,-0.171879,-0.168815,-0.177528,-0.007090,-0.041215,-0.010872,-0.081325,-0.074309


In [46]:
train_images_efficent_net.to_csv('train_images_efficent_net.csv', index=False)

In [47]:
test_images_efficient_net.to_csv('test_images_efficient_net.csv', index=False)